# Hochschule Bonn-Rhein-Sieg
### Scientific Experimentation and Evaluation, SS18
### youBot placing experiment

submitted by: `Abhishek Padalkar, Max Mensing, Pranjal Dhole`

## Packages

In [1]:
import numpy as np
import pandas as pd
import os

## Importing data

In [28]:
def get_filepath(filename: str):
    PATH = os.getcwd()
    return PATH+'/recordings/CSVs/'+filename+'.csv'

In [29]:
load = ['small', 'medium', 'big']
move = ['left', 'right', 'straight']

In [30]:
for i in load:
    for j in move:
        filename = i+'_'+j
        print(filename)

small_left
small_right
small_straight
medium_left
medium_right
medium_straight
big_left
big_right
big_straight


In [31]:
big_left = np.loadtxt(get_filepath('big_left'))
display(big_left)

array([[-109.2734, -101.337 ,   -2.0878],
       [-106.4432,  -99.0096,   -2.1088],
       [-106.7756,  -99.332 ,   -2.0976],
       [-106.719 ,  -99.3026,   -2.093 ],
       [-106.7042,  -99.2226,   -2.1174],
       [-106.7024,  -99.2204,   -2.1146],
       [-107.784 , -100.2024,   -2.1   ],
       [-107.7904, -100.1474,   -2.1046],
       [-106.3724,  -99.008 ,   -2.0904],
       [-107.4476,  -99.872 ,   -2.1096],
       [-107.41  ,  -99.89  ,   -2.1   ],
       [-107.136 ,  -99.751 ,   -2.0788],
       [-107.3676,  -99.854 ,   -2.0996],
       [-106.4404,  -99.122 ,   -2.1096],
       [-106.3926,  -99.0264,   -2.0904],
       [-106.838 ,  -99.402 ,   -2.093 ],
       [-106.0348,  -98.7308,   -2.1144],
       [-106.6514,  -99.178 ,   -2.1172],
       [-107.8994, -100.2984,   -2.1002],
       [-106.35  ,  -98.99  ,   -2.09  ]])